In [4]:
#importing packages
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

# Import Graphs

In [5]:
#mannheim
mann = pd.read_csv("/pfs/work9/workspace/scratch/ma_tofuchs-GraphWave-Seminar/Datasets/Mannheim/train_data/sensor_graph/adj_mx.csv", index_col = 0)
mann.index = mann.index.astype(str)
mann_adj = nx.from_pandas_adjacency(mann)

#la
with open("/pfs/work9/workspace/scratch/ma_tofuchs-GraphWave-Seminar/Datasets/Meta-LA/train_data/sensor_graph/adj_mx.pkl", "rb") as f:
    G = pickle.load(f, encoding="latin1")
la_adj = nx.from_numpy_array(G[2])

# Sparse Adj. Matrixies 

In [6]:
G = mann_adj  # e.g. nx.read_gpickle("mannheim_graph.gpickle")

# 2. Choose your Laplacian‐threshold τ
tau = 0.2

# 3. Fix a consistent node ordering
nodes = list(G.nodes())

# 4. Compute the normalized Laplacian matrix L (dense)
#    L[i,j] = -A[i,j]/sqrt(d_i d_j) for i≠j,  L[i,i]=1
L = nx.normalized_laplacian_matrix(G, nodelist=nodes).toarray()

# 5. Build a pruned copy of G
G_sparse_2 = nx.Graph()
G_sparse_2.add_nodes_from(G.nodes(data=True))

# 6. Re‐add only those edges whose |L_ij| ≥ τ
for i, u in enumerate(nodes):
    for j in range(i+1, len(nodes)):
        v = nodes[j]
        if G.has_edge(u, v):
            if abs(L[i, j]) >= tau:
                # preserve the original weight attribute if present
                w = G[u][v].get('weight', 1.0)
                G_sparse_2.add_edge(u, v, weight=w)

# 7. Inspect
print("Original:", G.number_of_nodes(), "nodes,", G.number_of_edges(), "edges")
print("Pruned:  ", G_sparse_2.number_of_nodes(), "nodes,", G_sparse_2.number_of_edges(), "edges")

Original: 25 nodes, 113 edges
Pruned:   25 nodes, 10 edges


In [7]:
G = mann_adj  # e.g. nx.read_gpickle("mannheim_graph.gpickle")

# 2. Choose your Laplacian‐threshold τ
tau = 0.1

# 3. Fix a consistent node ordering
nodes = list(G.nodes())

# 4. Compute the normalized Laplacian matrix L (dense)
#    L[i,j] = -A[i,j]/sqrt(d_i d_j) for i≠j,  L[i,i]=1
L = nx.normalized_laplacian_matrix(G, nodelist=nodes).toarray()

# 5. Build a pruned copy of G
G_sparse = nx.Graph()
G_sparse.add_nodes_from(G.nodes(data=True))

# 6. Re‐add only those edges whose |L_ij| ≥ τ
for i, u in enumerate(nodes):
    for j in range(i+1, len(nodes)):
        v = nodes[j]
        if G.has_edge(u, v):
            if abs(L[i, j]) >= tau:
                # preserve the original weight attribute if present
                w = G[u][v].get('weight', 1.0)
                G_sparse.add_edge(u, v, weight=w)

# 7. Inspect
print("Original:", G.number_of_nodes(), "nodes,", G.number_of_edges(), "edges")
print("Pruned:  ", G_sparse.number_of_nodes(), "nodes,", G_sparse.number_of_edges(), "edges")

Original: 25 nodes, 113 edges
Pruned:   25 nodes, 48 edges


In [8]:
#save the new adj matrixies 
tau_1_adj = nx.to_pandas_adjacency(G_sparse)
tau_1_adj.to_csv("adj_sparse_1.csv", index=True)

tau_2_adj = nx.to_pandas_adjacency(G_sparse_2)
tau_2_adj.to_csv("adj_sparse_2.csv", index=True)


# Graph Theory

In [9]:
# Replace these with your actual graphs
graphs = {
    "METR-LA": la_adj,
    "Mannheim": mann_adj, #,
    "Mannheim t=.2": G_sparse_2, 
    "Mannheim t=.1": G_sparse
    #"PEMS-BAY": pems_graph
}

def spectral_gap(G):
    A = nx.to_numpy_array(G)
    eigs = np.real(np.linalg.eigvals(A))
    eigs_sorted = np.sort(eigs)[::-1]
    return eigs_sorted[0] - eigs_sorted[1]

def algebraic_connectivity(G):
    # normalized Laplacian
    L = nx.normalized_laplacian_matrix(G).toarray()
    eigs = np.real(np.linalg.eigvals(L))
    eigs_sorted = np.sort(eigs)
    return eigs_sorted[1]  # Fiedler value

for name, G in graphs.items():
    V = G.number_of_nodes()
    E = G.number_of_edges()
    avg_deg = 2 * E / V
    avg_clust = nx.average_clustering(G, weight='weight')
    gap = spectral_gap(G)
    fiedler = algebraic_connectivity(G)
    print(f"--- {name} ---")
    print(f" Nodes               : {V}")
    print(f" Edges               : {E}")
    print(f" Average degree      : {avg_deg:.2f}")
    print(f" Average clustering  : {avg_clust:.3f}")
    print(f" Spectral gap        : {gap:.3f}")
    print(f" Algebraic connectivity (Fiedler) : {fiedler:.3f}")
    print()

--- METR-LA ---
 Nodes               : 207
 Edges               : 1520
 Average degree      : 14.69
 Average clustering  : 0.221
 Spectral gap        : 1.643
 Algebraic connectivity (Fiedler) : 0.000

--- Mannheim ---
 Nodes               : 25
 Edges               : 113
 Average degree      : 9.04
 Average clustering  : 0.249
 Spectral gap        : 3.371
 Algebraic connectivity (Fiedler) : 0.153

--- Mannheim t=.2 ---
 Nodes               : 25
 Edges               : 10
 Average degree      : 0.80
 Average clustering  : 0.073
 Spectral gap        : 0.539
 Algebraic connectivity (Fiedler) : 0.000

--- Mannheim t=.1 ---
 Nodes               : 25
 Edges               : 48
 Average degree      : 3.84
 Average clustering  : 0.303
 Spectral gap        : 1.162
 Algebraic connectivity (Fiedler) : 0.091



def knn_prune(A: pd.DataFrame, k: int) -> pd.DataFrame:
    # Make an all-zero DataFrame of the same shape
    A_knn = pd.DataFrame(0, index=A.index, columns=A.columns)
    
    for u in A.index:
        # Sort neighbors of u by weight descending, pick top k (excluding self)
        topk = A.loc[u].nlargest(k+1).index.drop(u, errors='ignore')[:k]
        A_knn.loc[u, topk] = A.loc[u, topk]
    
    # Symmetrize: only keep edges if either direction selected
    A_sym = pd.DataFrame(
        np.maximum(A_knn.values, A_knn.values.T),
        index=A.index, columns=A.columns
    )
    return A_sym

# Example: keep each node’s top-5 edges
knn_sparse = knn_prune(mann, k=5)